Article https://towardsdatascience.com/dimensionality-reduction-for-data-visualization-pca-vs-tsne-vs-umap-be4aa7b1cb29 

PCA

In [1]:
import numpy as np
import pandas as pd
import time
# For plotting
import plotly.io as plt_io
import plotly.graph_objects as go
%matplotlib inline

In [2]:
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.1 MB/s 
     |████████████████████████████████| 1.1 MB 31.2 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=22fb641704a00bbacba2a91dd81a51266f1911104569f8d88bc5aeaf83ce850f
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=13f0c791783c675c503b466e68306157eb02e67d2870c5fb3177fd6b8f29e4cb
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built umap-learn pynndescent


In [3]:
#PCA
from sklearn.decomposition import PCA
#TSNE
from sklearn.manifold import TSNE
#UMAP
import umap.umap_ as umap
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [4]:
train=pd.read_csv("/content/sign_mnist_test.csv")

In [5]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,...,138.0,148.0,127.0,89.0,82.0,96.0,106.0,112.0,120.0,107.0
1,5,126,128,131,132,133,134,135,135,136,...,47.0,104.0,194.0,183.0,186.0,184.0,184.0,184.0,182.0,180.0
2,10,85,88,92,96,105,123,135,143,147,...,68.0,166.0,242.0,227.0,230.0,227.0,226.0,225.0,224.0,222.0
3,0,203,205,207,206,207,209,210,209,210,...,154.0,248.0,247.0,248.0,253.0,236.0,230.0,240.0,253.0,255.0
4,3,188,191,193,195,199,201,202,203,203,...,26.0,40.0,64.0,48.0,29.0,46.0,49.0,46.0,46.0,53.0


In [6]:
train=train[train["label"]<10]
y=train.loc[:,"label"].values
x=train.loc[:,"pixel1":].values

In [12]:
def plot_2d(component1, component2):
  fig = go.Figure(data=go.Scatter(
      x=component1,
      y=component2,
      mode="markers",
      marker=dict(
          size=20,
          color=y,
          colorscale="Rainbow",
          showscale=True,
          line_width=1
      )
  ))
  fig.update_layout(margin=dict(l=100, r=100, b=100, t=100), width=2000, height=1200)
  fig.layout.template="plotly_dark"
  fig.show()

In [13]:
def plot_3d(component1, component2, component3):
  fig=go.Figure(data=[go.Scatter3d(
      x=component1,
      y=component2,
      z=component3,
      mode="markers",
      marker=dict(
          size=10,
          color=y,
          colorscale="Rainbow",
          opacity=1,
          line_width=1
      )
  )])
  fig.update_layout(margin=dict(l=50,r=50,t=50), width=1800, height=1000)
  fig.layout.template = "plotly_dark"

  fig.show()

In [14]:
from sklearn.preprocessing import StandardScaler
x=StandardScaler().fit_transform(x)

In [15]:
start=time.time()
pca=PCA(n_components=3)
principalComponents = pca.fit_transform(x)
print("Duration:{}second".format(time.time()-start))
principal=pd.DataFrame(data=principalComponents
                       , columns= [ "principal component 1", "principal component 2", "principal component 3"])

Duration:0.10796308517456055second


In [16]:
plot_2d(principalComponents[:,0], principalComponents[:,1])

In [18]:
plot_3d(principalComponents[:, 0],principalComponents[:, 1],principalComponents[:, 2])

T-SNE

In [19]:
start=time.time()
pca_50=PCA(n_components=50)
pca_result_50=pca.fit_transform(x)
tsne=TSNE(random_state=42, n_components=3, verbose=0, perplexity=40,n_iter=400).fit_transform(pca_result_50)
print("Duration:{} seconds".format(time.time()-start))

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



Duration:25.531647205352783 seconds


In [20]:
plot_2d(tsne[:,0],tsne[:,1])

In [21]:
plot_3d(tsne[:,0],tsne[:,1], tsne[:,2])

UMAP

In [22]:
start=time.time()
reducer=umap.UMAP(random_state=42, n_components=3)
embedding=reducer.fit_transform(x)
print("Duration:{} seconds".format(time.time()-start))

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning:

The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.



Duration:22.26857018470764 seconds


In [23]:
plot_2d(reducer.embedding_[:,0], reducer.embedding_[:,1])

In [24]:
plot_3d(reducer.embedding_[:,0],reducer.embedding_[:,1],reducer.embedding_[:,2])

LDA

In [25]:
start=time.time()
X_LDA=LDA(n_components=3).fit_transform(x,y)
print("Duration:{} seconds".format(time.time()-start))

Duration:1.209571123123169 seconds


In [26]:
plot_2d(X_LDA[:,0], X_LDA[:,1])

In [27]:
plot_3d(X_LDA[:,0],X_LDA[:,1],X_LDA[:,2])